In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from datetime import datetime
from kmodes.kprototypes import KPrototypes
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


from pandas import ExcelWriter
from scipy.stats import zscore
from sklearn.cluster import KMeans
from matplotlib import style


from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [18]:
data = pd.read_csv('C:/Users/PL186042/Documents/data_with_cluster_updated2.csv')

In [19]:
data.head()

,TM_DRIVERID,TM_REGISTRATION_NUMBER,COMPANY_CODE,VEHICLE_TYPE_DESCRIPTION,PRODUCT_CODE,FUEL_TYPE,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,...,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy,cluster
0,65535,DN28LYGP,KFS,Other,FML,PETROL,150,20,30,155,...,0.0,359,2416,528,720528,38.26494,5,10,9,0
1,65535,DN28LYGP,KFS,PASS ALL(EXCL D/CAB),FML,PETROL,124,5,5,0,...,0.0,0,0,2760,9960,90.00000,9,11,10,2
2,65535,HKX536EC,KFS,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,162,4,9,162,...,0.0,376,2506,2500,204100,55.65427,12,15,11,2
3,65535,HKX536EC,KFS,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,161,26,19,119,...,0.0,284,1863,432,184032,46.48293,7,10,9,0
4,65535,HKX536EC,KFS,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,171,25,35,642,...,0.0,1638,9844,724,220324,52.07689,6,6,8,0


In [14]:
data.describe()

,TM_DRIVERID,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,NO_EXCESS_IDLE,NO_OF_TRIPS,ENGINE_CAPACITY,FUEL_CONSUMPTION,NUMBER_OF_GEARS,...,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy,cluster
count,24228.000000,24228.000000,24228.000000,24228.000000,24228.000000,24228.000000,24228.000000,24228.000000,24228.000000,24228.000000,...,24228.000000,24228.000000,24228.000000,24228.000000,2.422800e+04,24228.000000,24228.000000,24228.000000,24228.000000,24228.000000
mean,13909.909361,119.359790,11.983366,12.884927,117.462151,51.382079,157.071034,2254.305349,8.300799,5.328073,...,2049.397428,221.268408,1812.476845,1755.115032,1.908688e+05,76.650506,8.509823,10.743190,10.231633,1.102691
std,22711.208307,34.093766,23.273803,27.889365,336.521925,217.114679,182.718020,1507.608318,4.399166,1.038134,...,8870.144266,712.539282,5120.540935,1037.425887,2.712693e+05,20.436162,7.315166,7.630804,7.573637,1.087468
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,4.000000e+00,2.800638,0.000000,0.000000,0.000000,0.000000
25%,2907.000000,103.000000,0.000000,0.000000,0.000000,0.000000,24.000000,1496.000000,6.600000,5.000000,...,0.000000,0.000000,0.000000,852.000000,3.332200e+04,59.102792,5.000000,7.000000,7.000000,0.000000
50%,5549.000000,128.000000,4.000000,2.000000,0.000000,0.000000,117.000000,1987.000000,8.100000,5.000000,...,0.000000,0.000000,8.000000,1752.000000,1.522220e+05,83.791300,8.000000,10.000000,10.000000,1.000000
75%,6766.000000,141.000000,13.000000,13.000000,43.000000,15.000000,229.000000,2494.000000,8.700000,6.000000,...,0.000000,61.000000,708.000000,2648.000000,2.841690e+05,94.159290,11.000000,13.000000,13.000000,2.000000
max,65535.000000,250.000000,289.000000,300.000000,3000.000000,7088.000000,4359.000000,15928.000000,52.000000,8.000000,...,292810.520000,9792.000000,45361.000000,3596.000000,2.806104e+07,100.117198,100.000000,100.000000,100.000000,4.000000


In [42]:
Datadata[(data['TM_DRIVERID'] == 65535) &  (data['COMPANY_CODE'] == 'RSA') & (data['TM_REGISTRATION_NUMBER'] == '(NVL189) CA734057')]

In [32]:
Result = data[(data['TM_DRIVERID'] == 65535) &  (data['COMPANY_CODE'] == 'RSA')]

In [37]:
Result2 = Result.groupby(['TM_REGISTRATION_NUMBER','TM_DRIVERID'])['TM_REGISTRATION_NUMBER'].count()

In [43]:
pd.set_option('display.max_column', 999)


In [44]:
data[(data['TM_DRIVERID'] == 65535) &  (data['COMPANY_CODE'] == 'RSA') & (data['TM_REGISTRATION_NUMBER'] == '(NVL189) CA734057')]

,TM_DRIVERID,TM_REGISTRATION_NUMBER,COMPANY_CODE,VEHICLE_TYPE_DESCRIPTION,PRODUCT_CODE,FUEL_TYPE,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,NO_EXCESS_IDLE,NO_OF_TRIPS,ENGINE_CAPACITY,FUEL_CONSUMPTION,NUMBER_OF_GEARS,RENTAL,KM_DRIVEN,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy,cluster
19004,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,134,74,9,1,0,561,2494,9.6,5.0,6307.22,6592.900,17633.95,1,28,2260,686260,93.27724,12,20,20,4
19008,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,134,22,11,5,0,527,2494,9.6,5.0,6263.52,4307.491,17633.95,5,100,44,640844,95.31942,7,7,10,0
19013,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,134,2,2,3,46,34,2494,9.6,5.0,6263.52,165.225,17633.95,3,51,3356,24956,75.40750,10,15,10,2
19014,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,142,16,5,168,28,478,2494,9.6,5.0,6263.53,6291.900,17633.95,188,2568,1016,483416,78.88966,15,18,19,4
19114,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,93,2,0,0,0,65,2494,9.6,5.0,5914.86,222.900,17633.95,0,0,3500,50300,89.46576,10,14,15,2
19115,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,133,13,5,1,99,233,2494,9.6,5.0,6263.52,4453.400,17633.95,1,62,2720,348320,87.35732,9,12,12,2
19116,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,164,8,6,500,35,237,2494,9.6,5.0,6263.52,4345.400,17633.95,1491,7595,380,335180,56.17270,4,6,9,0


In [ ]:
TM_DRIVERID

TM_REGISTRATION_NUMBER

COMPANY_CODE

VEHICLE_TYPE_DESCRIPTION

PRODUCT_CODE

FUEL_TYPE

MAX_SPEED

NO_OF_HARSH_BRAKES

NO_OF_HARSH_ACCELERATION

NO_OF_SPEED_VIOLATIONS

NO_EXCESS_IDLE

NO_OF_TRIPS

ENGINE_CAPACITY

FUEL_CONSUMPTION

NUMBER_OF_GEARS

RENTAL

KM_DRIVEN

PROJECTED_EXCESS_KM_CHARGE

SPEED_PENALTY_POINTS

SPEEDING_TIME_SECONDS

DRIVING_TIME

total_time_driven

TM_SCORE

Num_times_yawned

Num_times_Angry

Num_times_sleepy

In [50]:
for i in data.columns:
    print(i,': ',len(data[i].unique()))

TM_DRIVERID :  2350
TM_REGISTRATION_NUMBER :  2432
COMPANY_CODE :  7
VEHICLE_TYPE_DESCRIPTION :  13
PRODUCT_CODE :  3
FUEL_TYPE :  3
MAX_SPEED :  226
NO_OF_HARSH_BRAKES :  220
NO_OF_HARSH_ACCELERATION :  246
NO_OF_SPEED_VIOLATIONS :  1536
NO_EXCESS_IDLE :  928
NO_OF_TRIPS :  976
ENGINE_CAPACITY :  137
FUEL_CONSUMPTION :  113
NUMBER_OF_GEARS :  7
RENTAL :  2945
KM_DRIVEN :  18509
PROJECTED_EXCESS_KM_CHARGE :  606
SPEED_PENALTY_POINTS :  2127
SPEEDING_TIME_SECONDS :  5251
DRIVING_TIME :  903
total_time_driven :  20332
TM_SCORE :  15331
Num_times_yawned :  99
Num_times_Angry :  99
Num_times_sleepy :  101
cluster :  5


In [72]:
data.shape

(24228, 27)

In [58]:
data_new = data.groupby(['TM_DRIVERID','TM_REGISTRATION_NUMBER'])

In [59]:
print(data_new)

In [71]:
data_new1.shape

(7039, 18)

In [121]:
data_new1 = data.groupby(['TM_DRIVERID','TM_REGISTRATION_NUMBER' , 'COMPANY_CODE' , 'VEHICLE_TYPE_DESCRIPTION' , 'PRODUCT_CODE','FUEL_TYPE']).agg({'MAX_SPEED': 'max', 'NO_OF_HARSH_BRAKES': 'mean' ,'NO_OF_HARSH_ACCELERATION': 'mean',
                                   'NO_OF_SPEED_VIOLATIONS': 'mean','NO_EXCESS_IDLE' : 'mean' ,'NO_OF_TRIPS' : 'mean' , 'ENGINE_CAPACITY' : 'mean' ,'FUEL_CONSUMPTION' : 'mean' , 'NUMBER_OF_GEARS' : 'mean',
                                  'RENTAL' : 'mean' ,'KM_DRIVEN' : 'mean' ,'PROJECTED_EXCESS_KM_CHARGE' : 'mean', 'SPEED_PENALTY_POINTS' : 'mean' , 'SPEEDING_TIME_SECONDS' : 'mean',
                                   'DRIVING_TIME' : 'mean' ,'total_time_driven' : 'mean' ,'TM_SCORE' : 'mean' , 'Num_times_yawned' : 'mean' ,
                                    'Num_times_Angry' : 'mean' , 'Num_times_sleepy' : 'mean' })

In [122]:
data_new1.reset_index(inplace=True)

In [123]:
data_new1

,TM_DRIVERID,TM_REGISTRATION_NUMBER,COMPANY_CODE,VEHICLE_TYPE_DESCRIPTION,PRODUCT_CODE,FUEL_TYPE,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,NO_EXCESS_IDLE,NO_OF_TRIPS,ENGINE_CAPACITY,FUEL_CONSUMPTION,NUMBER_OF_GEARS,RENTAL,KM_DRIVEN,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy
0,0,HXW293MP,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,143,7.200000,13.000000,63.200000,41.200000,214.800000,2393,7.27,6.000000,12408.510000,2018.980000,0.00,83.600000,1009.400000,2021.600000,2.252216e+05,72.717186,8.800000,11.200000,10.800000
1,0,(CMT03) CA182160,RSA,TRUCK HEAVY,FM,DIESEL,97,0.000000,0.000000,0.000000,0.000000,35.800000,9726,0.00,5.332323,185.020000,1035.040000,0.00,0.000000,0.000000,2226.400000,3.607864e+05,93.123130,1.400000,2.200000,2.200000
2,0,(GETZ 116) CY71838,RSA,PASS ALL(EXCL D/CAB),FM,PETROL,152,8.200000,12.800000,22.800000,17.200000,130.600000,1599,9.96,5.000000,185.020000,1886.100000,0.00,36.400000,369.800000,1509.600000,1.663896e+05,79.880272,7.600000,10.400000,9.600000
3,0,(NVL188) CA347577,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,167,6.400000,0.600000,114.400000,1.000000,229.400000,2494,9.60,5.000000,6272.650000,3328.060000,11782.31,256.600000,1807.000000,2140.800000,2.260608e+05,77.504462,9.000000,11.400000,11.400000
4,0,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,162,5.800000,1.000000,84.400000,10.800000,275.400000,2494,9.60,5.000000,6263.530000,3987.120000,17633.95,137.400000,1301.800000,2150.400000,3.455904e+05,71.600656,10.600000,12.400000,11.800000
5,0,(NVL190) CA857305,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,163,13.600000,0.600000,645.000000,5.200000,229.600000,2494,9.60,5.000000,6798.188000,5637.580000,24321.79,1295.400000,9750.600000,2450.400000,4.452504e+05,59.714722,13.000000,17.400000,17.000000
6,0,(NVL203) CA726791,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,182,18.200000,9.000000,419.000000,410.400000,346.600000,2198,6.80,6.000000,6842.850000,4980.980000,8737.49,751.000000,6484.200000,2865.600000,6.091056e+05,64.834144,10.400000,13.200000,12.000000
7,0,(NVL205) CA724822,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,177,77.500000,138.000000,173.000000,293.500000,169.500000,2198,6.80,6.000000,9629.281353,2945.022500,14630.80,410.500000,2786.500000,1836.000000,3.708360e+05,50.408025,8.500000,11.000000,9.000000
8,0,(NVL208) CA498868,RSA,LIGHT COMMERCIAL VEHICLE,FML,PETROL,136,5.600000,4.000000,0.400000,14.200000,58.000000,1389,7.20,5.000000,4829.780000,543.500000,0.00,0.400000,8.400000,1816.800000,4.645680e+04,91.180816,7.800000,10.000000,9.600000
9,0,(NVL211) CA965233,RSA,PANEL VAN - MEDIUM,FML,DIESEL,137,8.250000,14.250000,1.250000,50.000000,159.750000,2500,8.30,5.000000,9443.410000,2336.150000,0.00,1.500000,22.250000,2437.000000,1.959370e+05,86.408968,9.750000,11.250000,12.250000


In [124]:
data_new1[(data_new1['TM_DRIVERID'] == 65535)  & (data_new1['TM_REGISTRATION_NUMBER'] == '(NVL189) CA734057')]

,TM_DRIVERID,TM_REGISTRATION_NUMBER,COMPANY_CODE,VEHICLE_TYPE_DESCRIPTION,PRODUCT_CODE,FUEL_TYPE,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,NO_EXCESS_IDLE,NO_OF_TRIPS,ENGINE_CAPACITY,FUEL_CONSUMPTION,NUMBER_OF_GEARS,RENTAL,KM_DRIVEN,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy
5765,65535,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,164,19.571429,5.428571,96.857143,29.714286,305.0,2494,9.6,5.0,6219.955714,3768.459429,17633.95,241.285714,1486.285714,1896.571429,367039.428571,82.269943,9.571429,13.142857,13.571429


In [125]:
data_new1['Num_times_Angry'] = data_new1['Num_times_Angry'].apply(lambda x : int(x))

In [126]:
data_new1['Num_times_yawned'] = data_new1['Num_times_yawned'].apply(lambda x : int(x))

In [127]:
data_new1['Num_times_sleepy'] = data_new1['Num_times_sleepy'].apply(lambda x : int(x))

In [128]:
data_new1

,TM_DRIVERID,TM_REGISTRATION_NUMBER,COMPANY_CODE,VEHICLE_TYPE_DESCRIPTION,PRODUCT_CODE,FUEL_TYPE,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,NO_EXCESS_IDLE,NO_OF_TRIPS,ENGINE_CAPACITY,FUEL_CONSUMPTION,NUMBER_OF_GEARS,RENTAL,KM_DRIVEN,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy
0,0,HXW293MP,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,143,7.200000,13.000000,63.200000,41.200000,214.800000,2393,7.27,6.000000,12408.510000,2018.980000,0.00,83.600000,1009.400000,2021.600000,2.252216e+05,72.717186,8,11,10
1,0,(CMT03) CA182160,RSA,TRUCK HEAVY,FM,DIESEL,97,0.000000,0.000000,0.000000,0.000000,35.800000,9726,0.00,5.332323,185.020000,1035.040000,0.00,0.000000,0.000000,2226.400000,3.607864e+05,93.123130,1,2,2
2,0,(GETZ 116) CY71838,RSA,PASS ALL(EXCL D/CAB),FM,PETROL,152,8.200000,12.800000,22.800000,17.200000,130.600000,1599,9.96,5.000000,185.020000,1886.100000,0.00,36.400000,369.800000,1509.600000,1.663896e+05,79.880272,7,10,9
3,0,(NVL188) CA347577,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,167,6.400000,0.600000,114.400000,1.000000,229.400000,2494,9.60,5.000000,6272.650000,3328.060000,11782.31,256.600000,1807.000000,2140.800000,2.260608e+05,77.504462,9,11,11
4,0,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,162,5.800000,1.000000,84.400000,10.800000,275.400000,2494,9.60,5.000000,6263.530000,3987.120000,17633.95,137.400000,1301.800000,2150.400000,3.455904e+05,71.600656,10,12,11
5,0,(NVL190) CA857305,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,163,13.600000,0.600000,645.000000,5.200000,229.600000,2494,9.60,5.000000,6798.188000,5637.580000,24321.79,1295.400000,9750.600000,2450.400000,4.452504e+05,59.714722,13,17,17
6,0,(NVL203) CA726791,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,182,18.200000,9.000000,419.000000,410.400000,346.600000,2198,6.80,6.000000,6842.850000,4980.980000,8737.49,751.000000,6484.200000,2865.600000,6.091056e+05,64.834144,10,13,12
7,0,(NVL205) CA724822,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,177,77.500000,138.000000,173.000000,293.500000,169.500000,2198,6.80,6.000000,9629.281353,2945.022500,14630.80,410.500000,2786.500000,1836.000000,3.708360e+05,50.408025,8,11,9
8,0,(NVL208) CA498868,RSA,LIGHT COMMERCIAL VEHICLE,FML,PETROL,136,5.600000,4.000000,0.400000,14.200000,58.000000,1389,7.20,5.000000,4829.780000,543.500000,0.00,0.400000,8.400000,1816.800000,4.645680e+04,91.180816,7,10,9
9,0,(NVL211) CA965233,RSA,PANEL VAN - MEDIUM,FML,DIESEL,137,8.250000,14.250000,1.250000,50.000000,159.750000,2500,8.30,5.000000,9443.410000,2336.150000,0.00,1.500000,22.250000,2437.000000,1.959370e+05,86.408968,9,11,12


In [129]:
def timeConversion(x):
    return x /3600.0


In [130]:
data_new1['total_time_driven'] = data_new1['total_time_driven'].apply(lambda x: timeConversion(x))

In [131]:
data_new1['total_time_driven'] = data_new1['total_time_driven'].apply(lambda x : int(x))

In [132]:
data_new1['NO_OF_HARSH_BRAKES'] = data_new1['NO_OF_HARSH_BRAKES'].apply(lambda x : int(x))
data_new1['NO_OF_HARSH_ACCELERATION'] = data_new1['NO_OF_HARSH_ACCELERATION'].apply(lambda x : int(x))
data_new1['NO_OF_SPEED_VIOLATIONS'] = data_new1['NO_OF_SPEED_VIOLATIONS'].apply(lambda x : int(x))
data_new1['NO_EXCESS_IDLE'] = data_new1['NO_EXCESS_IDLE'].apply(lambda x : int(x))
data_new1['NO_OF_TRIPS'] = data_new1['NO_OF_TRIPS'].apply(lambda x : int(x))
data_new1['NUMBER_OF_GEARS'] = data_new1['NUMBER_OF_GEARS'].apply(lambda x : int(x))

In [133]:
data_new1

,TM_DRIVERID,TM_REGISTRATION_NUMBER,COMPANY_CODE,VEHICLE_TYPE_DESCRIPTION,PRODUCT_CODE,FUEL_TYPE,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,NO_EXCESS_IDLE,NO_OF_TRIPS,ENGINE_CAPACITY,FUEL_CONSUMPTION,NUMBER_OF_GEARS,RENTAL,KM_DRIVEN,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy
0,0,HXW293MP,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,143,7,13,63,41,214,2393,7.27,6,12408.510000,2018.980000,0.00,83.600000,1009.400000,2021.600000,62,72.717186,8,11,10
1,0,(CMT03) CA182160,RSA,TRUCK HEAVY,FM,DIESEL,97,0,0,0,0,35,9726,0.00,5,185.020000,1035.040000,0.00,0.000000,0.000000,2226.400000,100,93.123130,1,2,2
2,0,(GETZ 116) CY71838,RSA,PASS ALL(EXCL D/CAB),FM,PETROL,152,8,12,22,17,130,1599,9.96,5,185.020000,1886.100000,0.00,36.400000,369.800000,1509.600000,46,79.880272,7,10,9
3,0,(NVL188) CA347577,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,167,6,0,114,1,229,2494,9.60,5,6272.650000,3328.060000,11782.31,256.600000,1807.000000,2140.800000,62,77.504462,9,11,11
4,0,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,162,5,1,84,10,275,2494,9.60,5,6263.530000,3987.120000,17633.95,137.400000,1301.800000,2150.400000,95,71.600656,10,12,11
5,0,(NVL190) CA857305,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,163,13,0,645,5,229,2494,9.60,5,6798.188000,5637.580000,24321.79,1295.400000,9750.600000,2450.400000,123,59.714722,13,17,17
6,0,(NVL203) CA726791,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,182,18,9,419,410,346,2198,6.80,6,6842.850000,4980.980000,8737.49,751.000000,6484.200000,2865.600000,169,64.834144,10,13,12
7,0,(NVL205) CA724822,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,177,77,138,173,293,169,2198,6.80,6,9629.281353,2945.022500,14630.80,410.500000,2786.500000,1836.000000,103,50.408025,8,11,9
8,0,(NVL208) CA498868,RSA,LIGHT COMMERCIAL VEHICLE,FML,PETROL,136,5,4,0,14,58,1389,7.20,5,4829.780000,543.500000,0.00,0.400000,8.400000,1816.800000,12,91.180816,7,10,9
9,0,(NVL211) CA965233,RSA,PANEL VAN - MEDIUM,FML,DIESEL,137,8,14,1,50,159,2500,8.30,5,9443.410000,2336.150000,0.00,1.500000,22.250000,2437.000000,54,86.408968,9,11,12


In [134]:
data_new1.to_csv('C:/Users/PL186042/Documents/Driver_action_assets/Emotion-Recognition-using-Python-master/Final_code/clean_data_telematic.csv', index = False)

In [135]:
numeric_cols = data_new1.select_dtypes(include=[np.number]).columns
data2 = data_new1[numeric_cols].apply(zscore)

In [136]:
#Setting for plot 
style.use("ggplot")
colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']

In [137]:
syms1 = np.array(data2['TM_DRIVERID'])

In [138]:
X1 = np.array(data2.iloc[:,1:])

In [139]:
kproto1 = KPrototypes(n_clusters=5, init='Cao', verbose=2)
clusters1 = kproto1.fit_predict(X1, categorical=[0,1,2,3,4])

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 2323, ncost: 89993.14887760436
Run: 1, iteration: 2/100, moves: 1060, ncost: 87623.62050884325
Run: 1, iteration: 3/100, moves: 695, ncost: 86478.54881945756
Run: 1, iteration: 4/100, moves: 354, ncost: 86195.99865940005
Run: 1, iteration: 5/100, moves: 274, ncost: 85959.95373210874
Run: 1, iteration: 6/100, moves: 214, ncost: 85801.03101918205
Run: 1, iteration: 7/100, moves: 210, ncost: 85577.59841141304
Run: 1, iteration: 8/100, moves: 267, ncost: 84801.04831606282
Run: 1, iteration: 9/100, moves: 579, ncost: 80783.73279923902
Run: 1, iteration: 10/100, moves: 984, ncost: 77000.53856852248
Run: 1, iteration: 11/100, moves: 555, ncost: 75310.18227880786
Run: 1, iteration: 12/100, moves: 217, ncost: 74847.13374471941
Run: 1, iteration: 13/100, moves: 72, ncost: 74607.09936528296
Run: 1, iteration: 1

In [141]:
# Adding the cluster column to the original data which will act as the label for the data 
df = pd.DataFrame(clusters1)
df.columns = ['cluster']
print(df.groupby(['cluster']).size())
data_new1['cluster'] = df['cluster']

cluster
0    4213
1    2198
2     133
3     424
4      72
dtype: int64


In [143]:
data_new1.head()

,TM_DRIVERID,TM_REGISTRATION_NUMBER,COMPANY_CODE,VEHICLE_TYPE_DESCRIPTION,PRODUCT_CODE,FUEL_TYPE,MAX_SPEED,NO_OF_HARSH_BRAKES,NO_OF_HARSH_ACCELERATION,NO_OF_SPEED_VIOLATIONS,NO_EXCESS_IDLE,NO_OF_TRIPS,ENGINE_CAPACITY,FUEL_CONSUMPTION,NUMBER_OF_GEARS,RENTAL,KM_DRIVEN,PROJECTED_EXCESS_KM_CHARGE,SPEED_PENALTY_POINTS,SPEEDING_TIME_SECONDS,DRIVING_TIME,total_time_driven,TM_SCORE,Num_times_yawned,Num_times_Angry,Num_times_sleepy,cluster
0,0,HXW293MP,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,143,7,13,63,41,214,2393,7.27,6,12408.51,2018.98,0.00,83.6,1009.4,2021.6,62,72.717186,8,11,10,1
1,0,(CMT03) CA182160,RSA,TRUCK HEAVY,FM,DIESEL,97,0,0,0,0,35,9726,0.00,5,185.02,1035.04,0.00,0.0,0.0,2226.4,100,93.123130,1,2,2,4
2,0,(GETZ 116) CY71838,RSA,PASS ALL(EXCL D/CAB),FM,PETROL,152,8,12,22,17,130,1599,9.96,5,185.02,1886.10,0.00,36.4,369.8,1509.6,46,79.880272,7,10,9,0
3,0,(NVL188) CA347577,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,167,6,0,114,1,229,2494,9.60,5,6272.65,3328.06,11782.31,256.6,1807.0,2140.8,62,77.504462,9,11,11,1
4,0,(NVL189) CA734057,RSA,LIGHT COMMERCIAL VEHICLE,FML,DIESEL,162,5,1,84,10,275,2494,9.60,5,6263.53,3987.12,17633.95,137.4,1301.8,2150.4,95,71.600656,10,12,11,1


In [145]:
data_new1.to_csv('C:/Users/PL186042/Documents/Driver_action_assets/Emotion-Recognition-using-Python-master/Updated/data_with_cluster_20181004.csv',sep=',', index = False)